In [1]:
import osrm
from googlesearch import search
import requests
from fake_headers import Headers
from bs4 import BeautifulSoup
from selenium import webdriver as wd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pathlib import Path
from tqdm import tqdm
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import random
import time
from pathlib import Path
import shutil
from copy import deepcopy

In [5]:
# Импорт необходимых библиотек
from dbfread import DBF
import pandas as pd
import chardet

# Определение пути к файлу
file_path = 'geodata/post_codes.dbf'

# Чтение первых 500 байт для определения кодировки
with open(file_path, 'rb') as file:
    raw_data = file.read(500)

# Определение кодировки с помощью chardet
encoding_info = chardet.detect(raw_data)
detected_encoding = encoding_info['encoding']

# Чтение DBF файла с использованием определенной кодировки
try:
    table = DBF(file_path, encoding=detected_encoding)
    df = pd.DataFrame(iter(table))  # Преобразование в DataFrame
except Exception as e:
    print(f"Ошибка при чтении DBF: {e}")

# Сохранение DataFrame в CSV
csv_path = "geodata/post_codes.csv"
try:
    df.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"Файл успешно сохранен: {csv_path}")
except Exception as e:
    print(f"Ошибка при сохранении CSV: {e}")


Файл успешно сохранен: geodata/post_codes.csv


In [6]:
cities = pd.read_csv('geodata/table_geo.csv')
cities.city = cities.city.apply(lambda x: x.lower())
cities

,city,region,geo
0,Абаза,Хакасия,"52.651662,90.088581"
1,Абакан,Хакасия,"53.721152,91.442396"
2,Абдулино,Оренбургская область,"53.683962,53.654989"
3,Абинск,Краснодарский край,"44.866237,38.151288"
4,Агидель,Башкортостан,"55.899043,53.933439"
...,...,...,...
1120,Ярославль,Ярославская область,"57.479957,39.752643"
1121,Ярцево,Смоленская область,"55.066123,32.688292"
1122,Ясногорск,Тульская область,"54.479293,37.693354"
1123,Ясный,Оренбургская область,"51.035711,59.874088"


In [25]:
post_codes = pd.read_csv('geodata/post_codes.csv')
post_codes.columns = list(map(str.lower, post_codes.columns.tolist()))
post_codes = post_codes[['index', 'region', 'autonom', 'area', 'city', 'city_1']]
for col in ['region', 'autonom', 'area', 'city', 'city_1']:
    post_codes[col] = post_codes[col].apply(lambda x: x.lower() if isinstance(x, str) else None)

In [26]:
post_codes

,index,region,autonom,area,city,city_1
0,101000,москва,None,None,None,None
1,101300,москва,None,None,None,None
2,101700,москва,None,None,None,None
3,101701,москва,None,марушкинское,шарапово,None
4,101702,москва,None,None,None,None
...,...,...,...,...,...,...
60243,918042,московская область,None,None,химки,None
60244,918336,санкт-петербург,None,None,шушары,None
60245,990516,москва,None,сосенское,коммунарка,None
60246,150067,ярославская область,None,None,ярославль,None


In [36]:
'уфа' in cities.city.values

True

In [66]:
not_parsed = set()
for city, region in zip(post_codes.city.values, post_codes.region.values):
    if (city is not None) and (city not in cities.city.values):
        if region is None:
            region = ''
        if city is None:
            city = ''
        not_parsed.add((city, region))

In [86]:
not_parsed_unique = np.unique(list(not_parsed), axis=0)

In [89]:
not_parsed_unique

array([['1-го отделения совхоза "краснопольский"', 'воронежская область'],
       ['1-го отделения совхоза "масловский"', 'воронежская область'],
       ['1-го отделения совхоза "новоусманский"', 'воронежская область'],
       ...,
       ['ёнский', 'мурманская область'],
       ['ёрмица', 'коми республика'],
       ['ёртом', 'коми республика']], dtype='<U53')

In [90]:
not_parsed_unique.shape[0], len(not_parsed)

(27157, 27157)

In [91]:
not_parsed_unique

array([['1-го отделения совхоза "краснопольский"', 'воронежская область'],
       ['1-го отделения совхоза "масловский"', 'воронежская область'],
       ['1-го отделения совхоза "новоусманский"', 'воронежская область'],
       ...,
       ['ёнский', 'мурманская область'],
       ['ёрмица', 'коми республика'],
       ['ёртом', 'коми республика']], dtype='<U53')

In [92]:
unique_addr = iter(not_parsed_unique)

In [94]:
' '.join(next(unique_addr))

'1-го отделения совхоза "масловский" воронежская область'